In [1]:
from data import Data
from dimension_reduction import PCADimensionReduction
from autoencoder import Autoencoder

from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

2022-11-26 19:43:39.569352: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_filepath = "data/SC_integration/counts_ctc_simulated_123_5k.tsv"
true_results_filepath = "data/SC_integration/ids_ctc_simulated_123_5k.tsv"
healthy_train_indices_filepath = "data/SC_integration/healthy_train_indices.npy"
healthy_test_indices_filepath = "data/SC_integration/healthy_test_indices.npy"
train_indices_filepath = "data/SC_integration/train_indices.npy"
test_indices_filepath = "data/SC_integration/test_indices.npy"

SEED = 42
FOLD_NUMBER = 3

# You can change these values to work better for models
CUT_BY_MAX_THRESHOLD = 4

In [3]:
data_object = Data(data_filepath, true_results_filepath)

train_data, test_data, train_true_results, test_true_results = data_object.load_train_test_split(train_indices_filepath, test_indices_filepath)

In [4]:
healthy_train_data, cancer_train_data, healthy_test_data, cancer_test_data = data_object.load_train_test_healthy_cancer_data()


In [5]:
def create_autoencoder(input_shape):
    model = Sequential()
    model.add(Dense(50, input_dim=input_shape, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(input_shape))
    opt = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

## Autoencoder

### Regular data

In [6]:
autoencoder = create_autoencoder(healthy_train_data.shape[1])
autoencoder.fit(healthy_train_data, healthy_train_data, batch_size=16, epochs=100)

2022-11-26 19:43:41.479310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-26 19:43:41.499484: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-26 19:43:41.499885: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-26 19:43:41.500693: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Epoch 1/200
95/95 [==============================] - 1s 1ms/step - loss: 0.1935
Epoch 2/200
43/95 [============>.................] - ETA: 0s - loss: 0.1443

2022-11-26 19:43:42.950565: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


95/95 [==============================] - 0s 1ms/step - loss: 0.1427
Epoch 3/200
95/95 [==============================] - 0s 1ms/step - loss: 0.1424
Epoch 4/200
95/95 [==============================] - 0s 1ms/step - loss: 0.1418
Epoch 5/200
95/95 [==============================] - 0s 1ms/step - loss: 0.1416
Epoch 6/200
95/95 [==============================] - 0s 1ms/step - loss: 0.1413
Epoch 7/200
95/95 [==============================] - 0s 1ms/step - loss: 0.1409
Epoch 8/200
95/95 [==============================] - 0s 1ms/step - loss: 0.1403
Epoch 9/200
95/95 [==============================] - 0s 1ms/step - loss: 0.1395
Epoch 10/200
95/95 [==============================] - 0s 1ms/step - loss: 0.1383
Epoch 11/200
95/95 [==============================] - 0s 1ms/step - loss: 0.1365
Epoch 12/200
95/95 [==============================] - 0s 1ms/step - loss: 0.1339
Epoch 13/200
95/95 [==============================] - 0s 1ms/step - loss: 0.1309
Epoch 14/200
95/95 [============================

In [7]:
autoencoder_object = Autoencoder(autoencoder, healthy_train_data, healthy_test_data, cancer_train_data, cancer_test_data)
autoencoder_object.display_average_rmse()

95/95 [==============================] - 0s 695us/step
Healthy train RMSE: 0.32810185970353806
Healthy test RMSE: 0.33492738871755867
Cancer train RMSE: 0.511448964272528
Cancer test RMSE: 0.5093977424583281
Threshold: 0.41977541198803303


In [8]:
autoencoder_object.display_regular_metrics(train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(test_data, test_true_results, "Test")

97/97 [==============================] - 0s 745us/step
Train balanced accuracy: 0.9732101725191558
Train AUC: 0.9732101725191558
Train f1 score: 0.46896551724137925
Train precision: 0.3090909090909091
Train recall: 0.9714285714285714
42/42 [==============================] - 0s 695us/step
Test balanced accuracy: 0.9846508058326937
Test AUC: 0.9846508058326937
Test f1 score: 0.42857142857142855
Test precision: 0.2727272727272727
Test recall: 1.0


### Scaled data

In [9]:
scaled_healthy_train_data, scaled_healthy_test_data, scaled_cancer_train_data, scaled_cancer_test_data = data_object.get_scaled_healthy_train_test_cancer_data()
autoencoder = create_autoencoder(scaled_healthy_train_data.shape[1])
autoencoder.fit(scaled_healthy_train_data, scaled_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/200
95/95 [==============================] - 0s 1ms/step - loss: 0.9344
Epoch 2/200
95/95 [==============================] - 0s 1ms/step - loss: 0.9094
Epoch 3/200
95/95 [==============================] - 0s 1ms/step - loss: 0.9013
Epoch 4/200
95/95 [==============================] - 0s 1ms/step - loss: 0.8958
Epoch 5/200
95/95 [==============================] - 0s 1ms/step - loss: 0.8909
Epoch 6/200
95/95 [==============================] - 0s 1ms/step - loss: 0.8868
Epoch 7/200
95/95 [==============================] - 0s 1ms/step - loss: 0.8840
Epoch 8/200
95/95 [==============================] - 0s 1ms/step - loss: 0.8817
Epoch 9/200
95/95 [==============================] - 0s 1ms/step - loss: 0.8800
Epoch 10/200
95/95 [==============================] - 0s 1ms/step - loss: 0.8784
Epoch 11/200
95/95 [==============================] - 0s 1ms/step - loss: 0.8770
Epoch 12/200
95/95 [==============================] - 0s 1ms/step - loss: 0.8755
Epoch 13/200
95/95 [=================

In [10]:
autoencoder_object = Autoencoder(autoencoder, scaled_healthy_train_data, scaled_healthy_test_data, scaled_cancer_train_data, scaled_cancer_test_data)
autoencoder_object.display_average_rmse()

95/95 [==============================] - 0s 669us/step
Healthy train RMSE: 0.8964297384155379
Healthy test RMSE: 0.9422563838391635
Cancer train RMSE: 7.536551092630457
Cancer test RMSE: 7.046494711656883
Threshold: 4.216490415522998


In [11]:
scaled_train_data = data_object.scaled_data_by_healthy_train(train_data)
scaled_test_data = data_object.scaled_data_by_healthy_train(test_data)

autoencoder_object.display_regular_metrics(scaled_train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(scaled_test_data, test_true_results, "Test")

97/97 [==============================] - 0s 701us/step
Train balanced accuracy: 0.9714285714285714
Train AUC: 0.9714285714285714
Train f1 score: 0.9705882352941176
Train precision: 1.0
Train recall: 0.9428571428571428
42/42 [==============================] - 0s 823us/step
Test balanced accuracy: 0.9984650805832693
Test AUC: 0.9984650805832694
Test f1 score: 0.8823529411764706
Test precision: 0.7894736842105263
Test recall: 1.0


### Cut by max data

In [12]:
cut_by_max_healthy_train_data, cut_by_max_healthy_test_data, cut_by_max_cancer_train_data, cut_by_max_cancer_test_data = data_object.get_cut_by_max_healthy_train_test_cancer_data(CUT_BY_MAX_THRESHOLD)
autoencoder = create_autoencoder(cut_by_max_healthy_train_data.shape[1])
autoencoder.fit(cut_by_max_healthy_train_data, cut_by_max_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/200
95/95 [==============================] - 0s 919us/step - loss: 2.3833
Epoch 2/200
95/95 [==============================] - 0s 934us/step - loss: 0.6009
Epoch 3/200
95/95 [==============================] - 0s 925us/step - loss: 0.5369
Epoch 4/200
95/95 [==============================] - 0s 923us/step - loss: 0.4937
Epoch 5/200
95/95 [==============================] - 0s 924us/step - loss: 0.4578
Epoch 6/200
95/95 [==============================] - 0s 981us/step - loss: 0.4347
Epoch 7/200
95/95 [==============================] - 0s 978us/step - loss: 0.4153
Epoch 8/200
95/95 [==============================] - 0s 922us/step - loss: 0.4000
Epoch 9/200
95/95 [==============================] - 0s 930us/step - loss: 0.3901
Epoch 10/200
95/95 [==============================] - 0s 919us/step - loss: 0.3839
Epoch 11/200
95/95 [==============================] - 0s 915us/step - loss: 0.3794
Epoch 12/200
95/95 [==============================] - 0s 923us/step - loss: 0.3759
Epoch 13/200


In [13]:
autoencoder_object = Autoencoder(autoencoder, cut_by_max_healthy_train_data, cut_by_max_healthy_test_data, cut_by_max_cancer_train_data, cut_by_max_cancer_test_data)
autoencoder_object.display_average_rmse()

95/95 [==============================] - 0s 555us/step
Healthy train RMSE: 0.5479278558529904
Healthy test RMSE: 0.5605656074481054
Cancer train RMSE: 0.7239787029911824
Cancer test RMSE: 0.8085683364086085
Threshold: 0.6359532794220863


In [14]:
cut_by_max_train_data = data_object.cut_by_max_healthy_data(train_data, CUT_BY_MAX_THRESHOLD)
cut_by_max_test_data = data_object.cut_by_max_healthy_data(test_data, CUT_BY_MAX_THRESHOLD)

autoencoder_object.display_regular_metrics(cut_by_max_train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(cut_by_max_test_data, test_true_results, "Test")

97/97 [==============================] - 0s 569us/step
Train balanced accuracy: 0.6769896112443003
Train AUC: 0.6769896112443002
Train f1 score: 0.06050955414012739
Train precision: 0.03204047217537943
Train recall: 0.5428571428571428
42/42 [==============================] - 0s 553us/step
Test balanced accuracy: 0.8553850089536966
Test AUC: 0.8553850089536966
Test f1 score: 0.08777429467084638
Test precision: 0.046052631578947366
Test recall: 0.9333333333333333
